In [ ]:
import os
from PIL import Image

# Replace with your training data path
data_dir = 'cats-vs-dogs/train'

for class_folder in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_folder)
    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        try:
            img = Image.open(img_path)
            img.verify()  # Will raise an exception if corrupted
        except (IOError, SyntaxError) as e:
            print(f"Removing corrupted image: {img_path}")
            os.remove(img_path)


In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "2"  # Limit number of threads
os.environ["TF_NUM_INTRAOP_THREADS"] = "2"
os.environ["TF_NUM_INTEROP_THREADS"] = "2"

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt

# 1. Create training and validation data generators
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    'dogs-vs-cats/train',
    target_size=(150, 150),
    batch_size=8,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    'dogs-vs-cats/train',
    target_size=(150, 150),
    batch_size=8,
    class_mode='binary',
    subset='validation'
)

# 2. Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# 3. Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 4. Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)

# 5. Plot the training results
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.legend()
plt.title("Model Accuracy Over Epochs")
plt.show()

# Optional: Save model for future use
model.save('my_model.keras')


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load your trained model
model = load_model("my_model.keras")  # Update path if needed

# Function to predict image
def predict_image(model, img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    confidence = prediction[0][0]
    label = "Dog" if confidence > 0.5 else "Cat"
    confidence_percent = confidence if confidence > 0.5 else 1 - confidence

    # Output result
    print(f"Prediction: {label} ({confidence_percent:.2%} confidence)")

# Run prediction on your image
predict_image(model, "image.png")  # Replace with your image path
